# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [1]:
import numpy as np
from scipy.stats import chisquare  # One-way chi square test

# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation

ind_obs = np.array([[1, 1], [2, 2]]).T
print(ind_obs)
print(chisquare(ind_obs, axis=None))

dep_obs = np.array([[16, 18, 16, 14, 12, 12], [32, 24, 16, 28, 20, 24]]).T
print(dep_obs)
print(chisquare(dep_obs, axis=None))

[[1 2]
 [1 2]]
Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)
[[16 32]
 [18 24]
 [16 16]
 [14 28]
 [12 20]
 [12 24]]
Power_divergenceResult(statistic=23.31034482758621, pvalue=0.015975692534127565)


In [2]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=38.69323106073592, pvalue=3.961609200867749e-09)


In [3]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Live Lecture - let's explore some more of scipy.stats

In [0]:
# Taking requests! Come to lecture with a topic or problem and we'll try it.

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.

In [1]:
!ls

drugsCom_raw.zip
LS_DS_142_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb


In [2]:
!unzip drugsCom_raw.zip

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [3]:
!ls

drugsCom_raw.zip
drugsComTest_raw.tsv
drugsComTrain_raw.tsv
LS_DS_142_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb


In [39]:
import pandas as pd
import scipy as stats
import numpy as np

df = pd.read_table('drugsComTest_raw.tsv')
df.describe()

,Unnamed: 0,rating,usefulCount
count,53766.000000,53766.000000,53766.000000
mean,116386.701187,6.976900,27.989752
std,67017.739881,3.285207,36.172833
min,0.000000,1.000000,0.000000
25%,58272.500000,4.000000,6.000000
50%,116248.500000,8.000000,16.000000
75%,174586.750000,10.000000,36.000000
max,232284.000000,10.000000,949.000000


In [11]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [17]:
type(df['drugName'])

pandas.core.series.Series

In [37]:
depression_drugName = df.groupby(['drugName','condition']).groups
print(depression_drugName)
depression_drugName['Mirtazapine', 'Depression']

{('Mirtazapine', 'Depression'): Int64Index([    0,   366,   742,  1996,  2172,  2225,  2251,  2992,  3108,
             3484,  3801,  4435,  5866,  8729, 10227, 12629, 13446, 13510,
            14389, 16971, 17136, 17376, 18532, 18783, 19221, 19450, 19972,
            21095, 21363, 22305, 22548, 24334, 24674, 25596, 25821, 27080,
            27733, 28124, 30515, 31937, 32154, 32318, 32579, 34052, 34178,
            35445, 36250, 36528, 38025, 38330, 38784, 40808, 40909, 41758,
            41884, 41937, 42899, 44134, 44985, 45106, 45518, 45839, 46047,
            46123, 47106, 47476, 47717, 47932, 48877, 48969, 49308, 50020,
            50726, 52932, 53221],
           dtype='int64'), ('Mesalamine', "Crohn's Disease, Maintenance"): Int64Index([    1,  1612,  2387,  9153, 11723, 12275, 13468, 13623, 18458,
            19903, 21902, 23668, 27356, 30957, 32498, 35727, 42662, 49501,
            51630],
           dtype='int64'), ('Bactrim', 'Urinary Tract Infection'): Int64Index([    2,  18

Int64Index([    0,   366,   742,  1996,  2172,  2225,  2251,  2992,  3108,
             3484,  3801,  4435,  5866,  8729, 10227, 12629, 13446, 13510,
            14389, 16971, 17136, 17376, 18532, 18783, 19221, 19450, 19972,
            21095, 21363, 22305, 22548, 24334, 24674, 25596, 25821, 27080,
            27733, 28124, 30515, 31937, 32154, 32318, 32579, 34052, 34178,
            35445, 36250, 36528, 38025, 38330, 38784, 40808, 40909, 41758,
            41884, 41937, 42899, 44134, 44985, 45106, 45518, 45839, 46047,
            46123, 47106, 47476, 47717, 47932, 48877, 48969, 49308, 50020,
            50726, 52932, 53221],
           dtype='int64')

In [27]:
rating_mean = df.groupby(['drugName'])['rating'].mean()
rating_mean.head(15)

drugName
A / B Otic                              10.000000
Abacavir / dolutegravir / lamivudine     9.000000
Abacavir / lamivudine                   10.000000
Abatacept                                6.500000
Abilify                                  6.149660
Abilify Discmelt                         9.000000
Abilify Maintena                        10.000000
Abiraterone                              9.666667
AbobotulinumtoxinA                      10.000000
Abreva                                   6.446809
Absorica                                 6.500000
Abstral                                  4.000000
Acamprosate                              8.764706
Acanya                                   7.000000
Acarbose                                 8.000000
Name: rating, dtype: float64

In [28]:
rating_stddev = df.groupby('drugName')['rating'].std()
rating_stddev.head(15)

drugName
A / B Otic                                   NaN
Abacavir / dolutegravir / lamivudine    2.169305
Abacavir / lamivudine                   0.000000
Abatacept                               3.209361
Abilify                                 3.521662
Abilify Discmelt                             NaN
Abilify Maintena                             NaN
Abiraterone                             0.577350
AbobotulinumtoxinA                           NaN
Abreva                                  3.871908
Absorica                                3.535534
Abstral                                      NaN
Acamprosate                             2.000891
Acanya                                  3.431877
Acarbose                                     NaN
Name: rating, dtype: float64

In [41]:
# pivotdf = pd.pivot_table(df,index=["drugName"],
#               values=["rating"],
#               aggfunc=[np.mean, confidence_interval])

[obtaining a confidence interval](https://stackoverflow.com/questions/28242593/correct-way-to-obtain-confidence-interval-with-scipy)